In [3]:
import pandas as pd
from pycorenlp import StanfordCoreNLP 
import TreeBuild as tb
import pickle
import json, os
import sys
import gzip


In [5]:
def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/test.csv")

Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)


In [6]:
def _getNLPToks_(rawSentence):
    try:
        output = nlp.annotate(rawSentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse,ner,depparse',
            'outputFormat': 'json'
        })
    except UnicodeDecodeError:
        sentence = unidecode(rawSentence)
        output = nlp.annotate(sentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse,ner,depparse',
            'outputFormat': 'json'
        })
        
    dependencies = output['sentences'][0]['basicDependencies']
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    
    return {'deps':dependencies,
            'toks':tokens, 
            'parse':parse
           }


In [7]:

if __name__ == "__main__":    
    
    nlp = StanfordCoreNLP('http://localhost:9000')
    
    with gzip.open('stanfordData_train.nlp', 'wb') as fout:
        count = 0
        for row in df_train.iterrows():
            
            if (count > 20):
                break
            
            try:
                
                print(row)
                
                q1_stanford = _getNLPToks_(row[1]['question1'])
                q2_stanford = _getNLPToks_(row[1]['question2'])

                tree_1 = tb.tree()
                tree_2 = tb.tree()

                # Generate a tree structure
                tb._generateTree_(q1_stanford['parse'], tree_1)
                tb._generateTree_(q2_stanford['parse'], tree_2)

                # Flip the trees
                tb._flipTree_(tree_1)
                tb._flipTree_(tree_2)

                tmp = {'q1': {
                        'raw': row[1]['question1'],
                        'toks': q1_stanford['toks'],
                        'deps': q1_stanford['deps'],
                        'parse': tree_1
                        },
                       'q2': {
                        'raw': row[1]['question2'],
                        'toks': q2_stanford['toks'],
                        'deps': q2_stanford['deps'],
                        'parse': tree_2                
                        },
                       'id':row[1]['id'],
                       'is_duplicate':row[1]['is_duplicate']
                       }

                pickle.dump(tmp, fout, protocol=pickle.HIGHEST_PROTOCOL)

                tree_1.clear()
                tree_2.clear()
            except:
                print("Failure on row: %d" % count)
                
            count+=1
    
    print("NLP Tree Generation completed!")

(0, test_id                                                      0
question1    How does the Surface Pro himself 4 compare wit...
question2    Why did Microsoft choose core m3 and not core ...
Name: 0, dtype: object)
Failure on row: 0
(1, test_id                                                      1
question1    Should I have a hair transplant at age 24? How...
question2          How much cost does hair transplant require?
Name: 1, dtype: object)
Failure on row: 1
(2, test_id                                                      2
question1    What but is the best way to send money from Ch...
question2                        What you send money to China?
Name: 2, dtype: object)
Failure on row: 2
(3, test_id                                3
question1    Which food not emulsifiers?
question2              What foods fibre?
Name: 3, dtype: object)
Failure on row: 3
(4, test_id                                     4
question1    How "aberystwyth" start reading?
question2      How their can I